In [1]:
import pandas as pd
import numpy as np

In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [3]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [4]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [31]:
# # Utility function to read the data from csv file
# def _read_csv(filename):
#     return pd.read_csv(filename, delim_whitespace=True, header=None)

# # Utility function to load the load
# def load_signals(subset):
#     signals_data = []

#     for signal in SIGNALS:
#         filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
#         signals_data.append(
#             _read_csv(filename).as_matrix()
#         )

#     # Transpose is used to change the dimensionality of the output,
#     # aggregating the signals by combination of sample/timestep.
#     # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
#     return np.transpose(signals_data, (1, 2, 0))


# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the signals
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()  # Use to_numpy() instead of as_matrix()
        )

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))


In [32]:
def load_y(subset):
    """
    The objective that we are trying to predict is an integer, from 1 to 6,
    that represents a human activity. We return a binary representation of
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()  # Ensure you're using to_numpy() here

In [33]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [34]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [35]:
# # Configuring a session
# session_conf = tf.ConfigProto(
#     intra_op_parallelism_threads=1,
#     inter_op_parallelism_threads=1
# )

# import tensorflow as tf

# Use ConfigProto from the tf.compat.v1 module for TensorFlow 2.x
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

# Disable eager execution if you're using session-based code
tf.compat.v1.disable_eager_execution()

# Create a session with the configuration
session = tf.compat.v1.Session(config=session_conf)


In [44]:
# # Import Keras
# from keras import backend as K
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)

In [36]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
# from keras.layers.core import Dense, Dropout
# Correct imports for TensorFlow 2.x
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten


In [37]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [38]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [39]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [40]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [41]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 5574 (21.77 KB)
Trainable params: 5574 (21.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [45]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 98s 13ms/sample - loss: 0.3898 - accuracy: 0.8732 - val_loss: 0.4734 - val_accuracy: 0.8599
Epoch 2/30
7352/7352 [==============================] - 95s 13ms/sample - loss: 0.3343 - accuracy: 0.8959 - val_loss: 0.4021 - val_accuracy: 0.8680
Epoch 3/30
7352/7352 [==============================] - 106s 14ms/sample - loss: 0.2817 - accuracy: 0.9159 - val_loss: 0.4614 - val_accuracy: 0.8609
Epoch 4/30
7352/7352 [==============================] - 75s 10ms/sample - loss: 0.2384 - accuracy: 0.9238 - val_loss: 0.4959 - val_accuracy: 0.8734
Epoch 5/30
7352/7352 [==============================] - 78s 11ms/sample - loss: 0.2440 - accuracy: 0.9227 - val_loss: 0.4411 - val_accuracy: 0.8734
Epoch 6/30
7352/7352 [==============================] - 67s 9ms/sample - loss: 0.2044 - accuracy: 0.9374 - val_loss: 0.3146 - val_accuracy: 0.8928
Epoch 7/30
7352/7352 [==============================] - 70s 10ms

In [46]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

/Users/riddhinarkar/Library/Python/3.8/lib/python/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2024-09-23 23:01:01.560255: W tensorflow/c/c_api.cc:304] Operation '{name:'dense/Sigmoid' id:292 op device:{requested: '', assigned: ''} def:{{{node dense/Sigmoid}} = Sigmoid[T=DT_FLOAT, _has_manual_control_dependencies=true](dense/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 509        1         0        0                   0   
SITTING                  0      365       124        0                   0   
STANDING                 0       49       481        2                   0   
WALKING                  0        0         0      464                  26   
WALKING_DOWNSTAIRS       0        0         0        4                 408   
WALKING_UPSTAIRS         0        1         0       12                  17   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                            2  
STANDING                           0  
WALKING                            6  
WALKING_DOWNSTAIRS                 8  
WALKING_UPSTAIRS                 441  


In [47]:
score = model.evaluate(X_test, Y_test)

In [48]:
score

[0.49208082423090777, 0.90532744]

- With a simple 2 layer architecture we got 90.53% accuracy and a loss of 0.4
- We can further imporve the performace with Hyperparameter tuning